# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
# Do all imports and installs here
import boto3
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import configparser

from pyspark.sql import SparkSession

# import utility

# import importlib
# importlib.reload(utility)
# from utility import visualise_missing_values, clean_immigration, clean_temperature_data
# from utility import clean_demographics_data

In [5]:
config = configparser.ConfigParser()
config.read_file(open('config.cfg'))
CAPSTONE_AWS_ACCESS_KEY_ID = config.get('capstone', 'AWS_ACCESS_KEY_ID')
CAPSTONE_AWS_SECRET_ACCESS_KEY = config.get('capstone', 'AWS_SECRET_ACCESS_KEY')
bucket_name=config.get('capstone', 'BUCKET')

In [6]:
s3_creds = {'region_name':"us-east-1",
            'aws_access_key_id': CAPSTONE_AWS_ACCESS_KEY_ID,
            'aws_secret_access_key': CAPSTONE_AWS_SECRET_ACCESS_KEY}
                          
client = boto3.client('s3', **s3_creds)
resource = boto3.resource('s3', **s3_creds)
bucket = resource.Bucket(bucket_name)

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [7]:
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport().getOrCreate()

df_immigration =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

print('{:,} rows in immigration file'.format(df_immigration.count()))

3,096,313 rows in immigration file


In [8]:
#write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

In [9]:
df_immigration.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [10]:
df_immigration.select('visapost').dropDuplicates().show(10)
df_immigration.select('airline').dropDuplicates().show(10)
df_immigration.select('visatype').dropDuplicates().show(10)

+--------+
|visapost|
+--------+
|     CRS|
|     KGL|
|     AKD|
|     BGM|
|     TRK|
|     MOR|
|     MDL|
|     KAT|
|     FRN|
|     CJT|
+--------+
only showing top 10 rows

+-------+
|airline|
+-------+
|     LT|
|     MM|
|     DZ|
|     CI|
|    447|
|    YFA|
|     TC|
|     AZ|
|     FI|
|    R0E|
+-------+
only showing top 10 rows

+--------+
|visatype|
+--------+
|      F2|
|     GMB|
|      B2|
|      F1|
|     CPL|
|      I1|
|      WB|
|      M1|
|      B1|
|      WT|
+--------+
only showing top 10 rows



In [11]:
df_immigration.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [12]:
#us cities demos
file_name = "us-cities-demographics.csv"
df_demographics = spark.read.csv(file_name, inferSchema=True, header=True, sep=';')

In [13]:
df_demographics.groupBy('State') \
    .agg({'Count':'sum'}) \
    .show()

+--------------------+----------+
|               State|sum(Count)|
+--------------------+----------+
|                Utah|   1180933|
|              Hawaii|    393445|
|           Minnesota|   1546985|
|                Ohio|   2651621|
|            Arkansas|    643597|
|              Oregon|   1665343|
|               Texas|  20029645|
|        North Dakota|    198197|
|        Pennsylvania|   2675056|
|         Connecticut|   1106528|
|            Nebraska|    809416|
|              Nevada|   2781296|
|         Puerto Rico|    997266|
|          Washington|   2832381|
|            Illinois|   5375987|
|            Oklahoma|   1729503|
|District of Columbia|    726519|
|            Delaware|     75048|
|              Alaska|    336228|
|          New Mexico|   1179529|
+--------------------+----------+
only showing top 20 rows



In [14]:
# display the first five records
df_demographics.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [15]:
#total size of dataframe
'{:,}'.format(df_demographics.count())

'2,891'

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [16]:
# list all files in the customer repository
files = os.listdir('../../data/18-83510-I94-Data-2016/')
files

['i94_apr16_sub.sas7bdat',
 'i94_sep16_sub.sas7bdat',
 'i94_nov16_sub.sas7bdat',
 'i94_mar16_sub.sas7bdat',
 'i94_jun16_sub.sas7bdat',
 'i94_aug16_sub.sas7bdat',
 'i94_may16_sub.sas7bdat',
 'i94_jan16_sub.sas7bdat',
 'i94_oct16_sub.sas7bdat',
 'i94_jul16_sub.sas7bdat',
 'i94_feb16_sub.sas7bdat',
 'i94_dec16_sub.sas7bdat']

In [25]:
##function to clean immigration data
def clean_immigration_file(df,clmns=None):
    drp_clmns=clmns
    df.df.drop(columns=drp_clmns)
    df=df.dropna(how='all')
    return df

In [27]:
##reading in all files --- resource intensive
immigration_df=pd.DataFrame()

for f in files:
    file_name= '../../data/18-83510-I94-Data-2016/' + f
    df = pd.read_sas(file_name, 'sas7bdat', encoding="latin-1")
    immigration_df = immigration_df.append(clean_immigration_file(df,['occup','entdepu','insnum']))
    print(f'{file_name} df.shape = ', immigration_df.shape)

KeyboardInterrupt: 

In [24]:
# View columns with missing data
pd.DataFrame(df_demographics.toPandas().isnull().sum())/df_demographics.count()

,0
City,0.000000
State,0.000000
Median Age,0.000000
Male Population,0.001038
Female Population,0.001038
Total Population,0.000000
Number of Veterans,0.004497
Foreign-born,0.004497
Average Household Size,0.005534
State Code,0.000000


In [18]:
df_demographics=df_demographics.dropna(subset=[
    'Male Population',
    'Female Population',
    'Number of Veterans',
    'Foreign-born',
    'Average Household Size'])
df_demographics = df_demographics.drop_duplicates(subset=['City', 'State', 'State Code', 'Race'])

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.